In [1]:
import pickle as pkl
import numpy as np
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt
import networkx as nx
import os.path as op
import os
import csv
import math

%matplotlib inline

In [ ]:
#also add in opening up all ndmg pickles

age = pkl.load(open('/Users/nalsharif/science/PING/PING_bids/ages.pkl', 'rb'))
behav = pkl.load(open('/Users/nalsharif/science/PING/PING_bids/behav.pkl', 'rb'))
subjids = pkl.load(open('/Users/nalsharif/science/PING/PING_bids/subjids.pkl', 'rb'))